# Mathematical model kinetics

In [1]:
#import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [20]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from vessel_manoeuvring_models.prime_system import PrimeSystem
from phd.visualization.plot_prediction import plot_total_force, plot_force_components, predict, plot_compare_model_forces, plot_parameter_contributions
from phd.visualization.bokeh_plotting import create_tab
from vessel_manoeuvring_models.symbols import *
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify, remove_functions
from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]
import statsmodels.api as sm
import sympy as sp
from sympy import Eq, symbols, latex
import paper
from paper import save_fig, save_eq

In [6]:
#%reload_kedro
ship='wPCC'
ship_data = catalog.load(f"{ship}.ship_data")
model_loaders = catalog.load(f"{ship}.models_VCT")

[01/08/24 15:31:01] INFO     Loading data from 'wPCC.ship_data' (YAMLDataSet)...                ]8;id=884902;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=191698;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    WARNING  /home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/par ]8;id=31444;file:///home/maa/anaconda3/envs/phd/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=601311;file:///home/maa/anaconda3/envs/phd/lib/python3.10/warnings.py#109\109]8;;\
                             titioned_dataset.py:200: KedroDeprecationWarning: 'PartitionedDataset'                
                             has been moved to `kedro-datasets` and will be removed in Kedro                       
                             0.19.0.                                                                               
                               warnings.warn(                                                                      
                                                                                                                   

                    WARNING  /home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/__i ]8;id=459519;file:///home/maa/anaconda3/envs/phd/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=390938;file:///home/maa/anaconda3/envs/phd/lib/python3.10/warnings.py#109\109]8;;\
                             nit__.py:44: KedroDeprecationWarning: 'AbstractDataSet' has been                      
                             renamed to 'AbstractDataset', and the alias will be removed in Kedro                  
                             0.19.0                                                                                
                               return getattr(kedro.io.core, name)                                                 
                                                                                                                   

                    INFO     Loading data from 'wPCC.models_VCT' (PartitionedDataset)...        ]8;id=609926;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=838961;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [7]:
model_loaders

{'semiempirical_covered': <bound method AbstractDataset.load of <phd.extras.datasets.model_dataset.ModularVesselSimulatorDataSet object at 0x7f4d68087040>>}

In [8]:
model = model_loaders['semiempirical_covered']()

In [9]:
model.X_eq

Eq(m*(\dot{u} - r**2*x_G - r*v), X_RHI + X_{\dot{u}}*\dot{u} + X_H(u, v, r, delta) + X_P(u, v, r, rev) + X_R(u, v, r, delta, thrust) + X_W(C_x3, C_x2, C_x0, C_x4, C_x5, aws, C_x1, A_XV, awa, rho_A))

In [22]:
eq_X = Eq(model.X_eq.lhs,X_D)
eq_Y = Eq(model.Y_eq.lhs,Y_D)
eq_N = Eq(model.N_eq.lhs,N_D)

save_eq(remove_functions(eq_X), file_name='X')
save_eq(remove_functions(eq_Y), file_name='Y')
save_eq(remove_functions(eq_N), file_name='N')

'mathematical_model_kinetics.X'

Eq(m*(\dot{u} - r**2*x_G - r*v), X_D)

''

'mathematical_model_kinetics.Y'

Eq(m*(\dot{r}*x_G + \dot{v} + r*u), Y_D)

''

'mathematical_model_kinetics.N'

Eq(I_z*\dot{r} + m*x_G*(\dot{v} + r*u), N_D)

''

In [23]:
save_eq(remove_functions(model.X_D_eq), file_name='X_D')
save_eq(remove_functions(model.Y_D_eq), file_name='Y_D')
save_eq(remove_functions(model.N_D_eq), file_name='N_D')

'mathematical_model_kinetics.X_D'

Eq(X_D, X_H + X_P + X_R + X_RHI + X_W)

''

'mathematical_model_kinetics.Y_D'

Eq(Y_D, Y_H + Y_P + Y_R + Y_RHI + Y_W)

''

'mathematical_model_kinetics.N_D'

Eq(N_D, N_H + N_P + N_R + N_RHI + N_W)

''